Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from Brillar Bank

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls())

data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Code\langchain_python\virtualenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Split the Loaded Data into Chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Get Embedding Model

In [4]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Instantiate Elasticsearch Store

In [5]:
from langchain_elasticsearch import ElasticsearchStore, SparseVectorStrategy, DenseVectorStrategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

dense_vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="dense_vector",
    embedding=embedding,
    strategy=DenseVectorStrategy()
)

sparse_vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="sparse_vector",
    strategy=SparseVectorStrategy(model_id=".elser_model_2"),
    bulk_kwargs={
      "request_timeout": 60
    }
)

Get LLM

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

Get Retriever

In [8]:
from langchain.retrievers import EnsembleRetriever

dense_retriever = dense_vector_store.as_retriever(search_kwargs={"k": 5})
sparse_retriever = sparse_vector_store.as_retriever(search_kwargs={"k": 5})

retriever = EnsembleRetriever(retrievers=[dense_retriever, sparse_retriever])

Create Chain

In [9]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [10]:
from helpers.conversation_retrieval_chain import invoke_conversational_retrieval_chain
from helpers.test_data import get_questions

questions = get_questions()
for question in questions:
  result = invoke_conversational_retrieval_chain(chain, question)
  print(f'{question}\n{result["answer"]}\n')

Tell me what is Brillar bank?
Brillar Bank offers a range of financial services and products, including various types of fixed deposit accounts, savings accounts, and investment options. It is dedicated to helping customers manage their finances, save for the future, and make investments. The bank also provides online and mobile banking services to facilitate convenient account management.

How many types of fixed deposit does Brillar Bank provide?
Brillar Bank provides six types of fixed deposit accounts: Fixed Deposit, Senior Savers Flexi Fixed Deposit, E-Fixed Deposit, Junior Fixed Deposit, Flexi Fixed Deposit, and Foreign Currency Fixed Deposit.

What are the interest rates for fixed deposit?
The interest rates for fixed deposits at Brillar Bank are as follows:

- 1 month: 2.15%
- 2 - 3 months: 2.25%
- 4 - 5 months: 2.30%
- 6 months: 2.30%
- 7 - 11 months: 2.35%
- 12 - 60 months: 2.35%

What are the interest rates for e-fixed deposit?
The interest rates for e-fixed deposits at Bril